# Train, Test sampling

In [1]:
import pandas as pd
import numpy as np

In [2]:
def Test_sampling(Dataset, indices_Short, indices_Long,  sample_num, seed):
    np.random.seed(seed)
    Short_sample = np.random.choice(indices_Short, sample_num, replace=False)
    Long_sample = np.random.choice(indices_Long, sample_num, replace=False)
    Dataset_sample = pd.concat([Dataset.iloc[list(Short_sample),:], Dataset.iloc[list(Long_sample),:]])
    Dataset_sample = Dataset_sample.reset_index()
    Dataset_sample = Dataset_sample.drop(columns=['index'])
    return Short_sample, Long_sample, Dataset_sample

In [3]:
Patent = pd.read_csv('Patent_final.csv')

In [4]:
Patent["lag_label"].tolist().index("Short")
indices_Short = [i for i, x in enumerate(Patent["lag_label"].tolist()) if x == "Short"]
indices_Long = [i for i, x in enumerate(Patent["lag_label"].tolist()) if x == "Long"]

In [5]:
Short_sample, Long_sample, Patent_test = Test_sampling(Patent, indices_Short, indices_Long,  500, 7214)
samples = list(Short_sample) + list(Long_sample)
Patent_train = Patent.loc[set(Patent.index) - set(samples)]
train_index = list(set(Patent.index) - set(samples))
Patent_train = Patent_train.reset_index()
Patent_train = Patent_train.drop(columns=['index'])

In [6]:
Patent_train_x = Patent_train.iloc[:,  2:27]
Patent_train_y = list(Patent_train['lag_label'])

Patent_test_x = Patent_test.iloc[:,  2:27]
Patent_test_x=(Patent_test_x-Patent_train_x.min())/(Patent_train_x.max()-Patent_train_x.min())  
Patent_train_x=(Patent_train_x-Patent_train_x.min())/(Patent_train_x.max()-Patent_train_x.min())  
Patent_test_y = list(Patent_test['lag_label'])

# Baseline models

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics    
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import re

In [8]:
def Evaluation(model, Test_x, Test_y):
    pred = model.predict(Test_x)
    fpr, tpr, thresholds = metrics.roc_curve(Test_y, model.predict_proba(Test_x)[:, 0], pos_label="Long")
    roc_auc = metrics.auc(fpr, tpr)
    print("Accuracy = {}".format(metrics.accuracy_score(Test_y, pred)))
    print("Precision = {}".format(metrics.precision_score(Test_y, pred, pos_label="Short")))
    print("Recall = {}".format(metrics.recall_score(Test_y, pred, pos_label="Short")))
    print("F1 score = {}".format(metrics.f1_score(Test_y, pred, pos_label="Short")))
    print("AUC = {}".format(roc_auc))

In [ ]:
### LR
lr_model = LogisticRegression()
lr_model.fit(Patent_train_x, Patent_train_y)
Evaluation(lr_model, Patent_test_x, Patent_test_y)

In [ ]:
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 5, 5), random_state=1)
mlp_model.fit(Patent_train_x, Patent_train_y)
Evaluation(mlp_model, Patent_test_x, Patent_test_y)

In [ ]:
### SVM
svm_model = SVC(C = 1, kernel = 'rbf',gamma = 0.1, probability = True)
svm_model.fit(Patent_train_x, Patent_train_y)
Evaluation(svm_model, Patent_test_x, Patent_test_y)

In [ ]:
### RF
rf_model = RandomForestClassifier(n_estimators=1000, max_depth = 3, max_features = 5, random_state = 748)
rf_model.fit(Patent_train_x, Patent_train_y)
Evaluation(rf_model, Patent_test_x, Patent_test_y)

In [9]:
xgb_model = xgb.XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth = 5, gamma = 0.1, seed = 1771)
xgb_model.fit(Patent_train_x, Patent_train_y)
Evaluation(xgb_model, Patent_test_x, Patent_test_y)

Accuracy = 0.653
Precision = 0.71671388101983
Recall = 0.506
F1 score = 0.5932004689331769
AUC = 0.7051999999999999


# Adj mat.

In [10]:
import scipy.sparse as sp


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


In [11]:
### Paper - Paper
Paper_Paper = pd.read_csv('Paper_Paper.csv')

unique_list = list(Paper_Paper['Appln_ID'])
unique_list = list(set(unique_list))
unique_list.sort()

orders = train_index + samples
Appln_ID = list(Patent.iloc[orders, :]['Appln_ID'])
Paper_Paper_adj = np.identity(len(Patent))


for x in unique_list:
    row_index = Appln_ID.index(x)
    cited_Paper_list = list(Paper_Paper.loc[Paper_Paper['Appln_ID'] == x, 'cited_Appln_ID'])
    for y in cited_Paper_list:
        col_index = Appln_ID.index(y)
        Paper_Paper_adj[row_index,col_index] = 1 / (len(cited_Paper_list)+1)
        Paper_Paper_adj[row_index,row_index] = 1 / (len(cited_Paper_list)+1)
    

### Paper - Applicant

Paper_Applicant = pd.read_csv('Paper_Applicant.csv')
Applicant_dictionary = pd.read_csv('Applicant_dictionary.csv')
Applicant_ID = list(Applicant_dictionary['Applicant_ID'])

unique_list = list(Paper_Applicant['Appln_ID'])
unique_list = list(set(unique_list))
unique_list.sort()

Paper_Applicant_adj = np.zeros([len(Patent),len(Applicant_ID)])

for x in unique_list:
    row_index = Appln_ID.index(x)
    Applicant_list = list(Paper_Applicant.loc[Paper_Applicant['Appln_ID'] == x, 'Applicant_ID'])
    for y in Applicant_list:
        col_index = Applicant_ID.index(y)
        Paper_Applicant_adj[row_index,col_index] = 1 /len(Applicant_list)

### Paper - Inventor
Paper_Inventor = pd.read_csv('Paper_Inventor.csv')
Inventor_dictionary = pd.read_csv('Inventor_dictionary.csv')
Inventor_ID = list(Inventor_dictionary['Inventor_ID'])

unique_list = list(Paper_Inventor['Appln_ID'])
unique_list = list(set(unique_list))
unique_list.sort()

Paper_Inventor_adj = np.zeros([len(Patent),len(Inventor_ID)])

for x in unique_list:
    row_index = Appln_ID.index(x)
    Inventor_list = list(Paper_Inventor.loc[Paper_Inventor['Appln_ID'] == x, 'Inventor_ID'])
    for y in Inventor_list:
        col_index = Inventor_ID.index(y)
        Paper_Inventor_adj[row_index,col_index] = 1 /len(Inventor_list)
        
### Paper - Field
Paper_Field = pd.read_csv('Paper_Field.csv')
Field_ID = list(Paper_Field['IPC'])
Field_ID = list(set(Field_ID))
Field_ID.sort()

unique_list = list(Paper_Field['Appln_ID'])
unique_list = list(set(unique_list))
unique_list.sort()

Paper_Field_adj = np.zeros([len(Patent),len(Field_ID)])

for x in unique_list:
    row_index = Appln_ID.index(x)
    Field_list = list(Paper_Field.loc[Paper_Field['Appln_ID'] == x, 'IPC'])
    for y in Field_list:
        col_index = Field_ID.index(y)
        Paper_Field_adj[row_index,col_index] = 1/len(Field_list)

        
### Co-Applicant
Applicant_Applicant = pd.read_csv('Applicant_Applicant.csv')
Applicant_Applicant['Weight'] = Applicant_Applicant['Count'] / Applicant_Applicant['total']

Applicant_Applicant_adj = np.identity(len(Applicant_ID))

for i in range(len(Applicant_Applicant)):
    temp_df = Applicant_Applicant.iloc[i,:]
    row_index = Applicant_ID.index(temp_df['Applicant_ID'])
    col_index = Applicant_ID.index(temp_df['Applicant2'])
    weight = temp_df['Weight']
    Applicant_Applicant_adj[row_index,col_index] = weight

### Co-Inventor
Inventor_Inventor = pd.read_csv('Inventor_Inventor.csv')
Inventor_Inventor['Weight'] = Inventor_Inventor['Count'] / Inventor_Inventor['total']

Inventor_Inventor_adj = np.identity(len(Inventor_ID))

for i in range(len(Inventor_Inventor)):
    temp_df = Inventor_Inventor.iloc[i,:]
    row_index = Inventor_ID.index(temp_df['Inventor_ID'])
    col_index = Inventor_ID.index(temp_df['Inventor2'])
    weight = temp_df['Weight']
    Inventor_Inventor_adj[row_index,col_index] = weight 
    

### Co-Field
Field_Field = pd.read_csv('Field_Field.csv')
Field_Field['Weight'] = Field_Field['Count'] / Field_Field['total']

Field_Field_adj = np.identity(len(Field_ID))

for i in range(len(Field_Field)):
    temp_df = Field_Field.iloc[i,:]
    row_index = Field_ID.index(temp_df['Field'])
    col_index = Field_ID.index(temp_df['Field2'])
    weight = temp_df['Weight']
    Field_Field_adj[row_index,col_index] = weight 

    
        
### Total adj
Total_adj = np.concatenate((Paper_Paper_adj, Paper_Applicant_adj, Paper_Inventor_adj, Paper_Field_adj), axis= 1)
temp_adj = np.concatenate((Paper_Applicant_adj.T, Applicant_Applicant_adj, np.zeros((len(Applicant_ID), len(Inventor_ID))), np.zeros((len(Applicant_ID), len(Field_ID)))), axis = 1)
Total_adj = np.concatenate((Total_adj, temp_adj), axis = 0)
temp_adj = np.concatenate((Paper_Inventor_adj.T, np.zeros((len(Inventor_ID), len(Applicant_ID))), Inventor_Inventor_adj, np.zeros((len(Inventor_ID), len(Field_ID)))), axis = 1)
Total_adj = np.concatenate((Total_adj, temp_adj), axis = 0)
temp_adj = np.concatenate((Paper_Field_adj.T, np.zeros((len(Field_ID), len(Applicant_ID))), np.zeros((len(Field_ID), len(Inventor_ID))), Field_Field_adj), axis = 1)
Total_adj = np.concatenate((Total_adj, temp_adj), axis = 0)




In [12]:
sum(Total_adj[4,:])

1.9999999999999998

# Node feature mat.

In [13]:
Node_feature = Patent.iloc[orders, 2:27]
Node_feature=(Node_feature-Node_feature.min())/(Node_feature.max()-Node_feature.min())  
Node_feature = Node_feature.values

Node_feature = np.concatenate((Node_feature, np.zeros((2413, 25))), axis = 0)
Node_feature = np.concatenate((Node_feature, np.concatenate((np.zeros((11849, 2413)), np.identity(2413)), axis = 0)), axis = 1)
def get_multiple_elements_in_list(in_list, in_indices):
    return [in_list[i] for i in in_indices]

### y값
temp_dummies = pd.get_dummies(Patent)
all_y = list(temp_dummies['lag_label_Short'])


train_index = list(set(Patent.index) - set(samples))

test_y = get_multiple_elements_in_list(all_y, samples)
train_y = get_multiple_elements_in_list(all_y, train_index)


In [14]:
Node_feature.shape

(14262, 2438)

In [15]:
Node_feature

array([[1.        , 0.39285714, 0.10714286, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.03571429, 0.03571429, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.05357143, 0.21428571, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

# GCN modeling

In [16]:
import argparse
import sys
import time
import copy

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

######## Model consturction
class GCNLayer(nn.Module):
    def __init__(self, in_dim, out_dim, act=None, bn=False):
        super(GCNLayer, self).__init__()
        
        self.use_bn = bn
        self.linear1 = nn.Linear(in_dim, out_dim)
        nn.init.xavier_uniform_(self.linear1.weight)
        self.linear2 = nn.Linear(in_dim, out_dim)
        nn.init.xavier_uniform_(self.linear2.weight)
        self.linear3 = nn.Linear(in_dim, out_dim)
        nn.init.xavier_uniform_(self.linear3.weight)
        self.linear4 = nn.Linear(in_dim, out_dim)
        nn.init.xavier_uniform_(self.linear4.weight)
        self.bn =nn.BatchNorm1d(out_dim)
        self.activation = act
        
    def forward(self, x, adj, c1, c2, c3):
        index = [i for i in range(0,(11849+c1))] + [i for i in range(12162,(12162+c2))] + [i for i in range(12871,(12871+c3))]
                
        out1 = self.linear1(x[0:11849, : ])
        out2 = self.linear2(x[11849:(11849+c1), : ])
        out3 = self.linear3(x[(11849+c1):(11849+c1+c2), : ])
        out4 = self.linear4(x[(11849+c1+c2):(11849+c1+c2+c3), : ])
        
        
        out1 = torch.matmul(adj[:, 0:11849], out1)
        out2 = torch.matmul(adj[:, 11849:(11849+c1)], out2)
        out3 = torch.matmul(adj[:, (11849+c1):(11849+c1+c2)], out3)
        out4 = torch.matmul(adj[:, (11849+c1+c2):(11849+c1+c2+c3)], out4)
        
        out = out1 + out2 + out3 + out4
        
        if self.use_bn:
            out = self.bn(out)
        if self.activation != None:
            out = self.activation(out)
        return out, adj

class SkipConnection(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(SkipConnection, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(in_dim, out_dim, bias = False)
        
    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        out = in_x + out_x
        return out


class Highway(nn.Module):
    
    def __init__(self, in_dim, out_dim):
        super(Highway, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(in_dim, out_dim, bias=False)
        self.linear_coef_in = nn.Linear(out_dim, out_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        z = self.gate_coefficient(in_x)
        out = torch.mul(z, out_x) + torch.mul(1.0-z, in_x)
        return out
            
    def gate_coefficient(self, in_x):
        x1 = self.linear_coef_in(in_x)
        return self.sigmoid(x1)



class GatedSkipConnection(nn.Module):
    
    def __init__(self, in_dim, out_dim):
        super(GatedSkipConnection, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(in_dim, out_dim, bias=False)
        self.linear_coef_in = nn.Linear(out_dim, out_dim)
        self.linear_coef_out = nn.Linear(out_dim, out_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        z = self.gate_coefficient(in_x, out_x)
        out = torch.mul(z, out_x) + torch.mul(1.0-z, in_x)
        return out
            
    def gate_coefficient(self, in_x, out_x):
        x1 = self.linear_coef_in(in_x)
        x2 = self.linear_coef_out(out_x)
        return self.sigmoid(x1+x2)



class GCNBlock(nn.Module):
    def __init__(self, n_layer, in_dim, hidden_dim, out_dim, bn = True, sc = 'sc'):
        super(GCNBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(n_layer):
            self.layers.append(GCNLayer(in_dim if i==0 else hidden_dim,
                                        hidden_dim if i==n_layer-1 else hidden_dim,
                                        nn.ReLU(),
                                        True if i!=n_layer-1 else False))
        if sc=='sc':
            self.sc = SkipConnection(in_dim, out_dim)
        elif sc == 'no':
            self.sc = None
        elif sc == 'gc':
            self.sc = GatedSkipConnection(in_dim, out_dim)
        elif sc == 'hw':
            self.sc = Highway(in_dim, out_dim)
        else:
            assert False, "Wrong sc type."
            
    def forward(self, x, adj, c1, c2, c3):
        residual = x
        for i, layer in enumerate(self.layers):
            out, adj = layer((x if i==0 else out), adj, c1, c2, c3)    
        if self.sc != None:
            out = self.sc(residual, out)
        return out, adj

            

class Classifier(nn.Module):
    def __init__(self, in_dim, out_dim, act = None):
        super(Classifier, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(self.in_dim, self.out_dim)
        nn.init.xavier_uniform_(self.linear.weight)
        self.activation = act
        
    def forward(self, x):
        out = self.linear(x)
        if self.activation != None:
            out = self.activation(out)
        return out

class GCNNet(nn.Module):
    def __init__(self, args):
        super(GCNNet, self).__init__()
        
        self.blocks = nn.ModuleList()
        for i in range(args.n_block):
            self.blocks.append(GCNBlock(args.n_layer,
                                        args.input_dim if i==0 else args.hidden_dim,
                                        args.hidden_dim,
                                        args.hidden_dim,   ### output dim 따로
                                        args.bn,
                                        args.sc))
        self.pred1 = Classifier(args.hidden_dim, args.pred_dim1, act = nn.ReLU())
        self.pred2 = Classifier(args.pred_dim1, args.pred_dim2,act = nn.ReLU())
        self.pred3 = Classifier(args.pred_dim2, args.pred_dim3,act = nn.ReLU())
        self.pred4 = Classifier(args.pred_dim3, 1,act = nn.Sigmoid())

    def forward(self, node_feature, adj, c1, c2, c3):
        for i, block in enumerate(self.blocks):
            out, adj = block((node_feature if i==0 else out), adj, c1, c2, c3)
        out = self.pred1(out)
        out = self.pred2(out)
        out = self.pred3(out)
        out = self.pred4(out)
        
        return out
        


        



# Train module

In [17]:
def train(epoch, args, criterion):
    t = time.time()
    net.train()
    optimizer.zero_grad()
    outputs = net(node_feature, Adj, c1, c2, c3)
    loss = criterion(outputs[0:10849], train_y.float())
    
    pred = (outputs > 0.5).float()
    pred_train = pred[0:10849]
    pred_test = pred[10849:11849]
    correct = (pred_train.transpose(0,1) == train_y.float()).float().sum()
    acc_train = correct / len(train_y)
    correct = (pred_test.transpose(0,1) == test_y.float()).float().sum()
    acc_test = correct / len(test_y)
    pred_list = pred_test.tolist()
    test_y_list = test_y.tolist()
    f1_score = metrics.f1_score(test_y_list, pred_list, pos_label= 1)
    precision = metrics.precision_score(test_y_list, pred_list, pos_label=1)
    recall = metrics.recall_score(test_y_list, pred_list, pos_label=1)
    outputs_list = outputs.tolist()
    fpr, tpr, thresholds = metrics.roc_curve(test_y_list, outputs_list[10849:11849], pos_label=1)

    roc_auc = metrics.auc(fpr, tpr)
    
    loss.backward()
    optimizer.step()
    
    print('Epoch: {:04d}'.format(epoch+1),
          'acc_train: {:.4f}'.format(acc_train),
          'acc_test: {:.4f}'.format(acc_test),
          'f1_test: {:.4f}'.format(f1_score),
          'precision: {:.4f}'.format(precision),
          'recall: {:.4f}'.format(recall)
)

# Main

In [18]:
import easydict
import numpy as np
import pandas as pd
import random
import sys
import scipy.sparse as sp
import torch

In [19]:
c1 = 313
c2 = 479
# 340
# 479
# 709
c3 = 431
#c3 = 431
#c3 = 835
#c3 = 1391
input_dim = 25 + c1 + c2 + c3

In [20]:
index = [i for i in range(0,(11849+c1))] + [i for i in range(12162,(12162+c2))] + [i for i in range(12871,(12871+c3))]
Total_adj1 = Total_adj[index, :][:, index]       


In [21]:
Total_adj1.shape

(13072, 13072)

In [22]:
Total_adj1

array([[0.14285714, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.1       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.11111111, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.3       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [23]:
index2 = [i for i in range(0,(25+c1))] + [i for i in range(338,(338+c2))] + [i for i in range(1047,(1047+c3))]
Node_feature1 = Node_feature[index, :][:, index2]

In [24]:
Node_feature1.shape

(13072, 1248)

In [25]:
input_dim

1248

In [26]:
args = easydict.EasyDict({ "seed": 714, "epochs": 1000, "lr": 0.0001, "weight_decay": 5e-4, "n_layer": 1, 
                                         "n_block": 2 , "input_dim": input_dim , "hidden_dim": 400 , "pred_dim1": 100, "pred_dim2": 25, 
                                         "pred_dim3": 25, "bn": True ,"sc": "gc","cuda": True})


In [27]:
torch.cuda.manual_seed(710)
torch.manual_seed(710)

In [28]:
### Seed: 347 Epoch: 584 acc_train: 0.7447 acc_test: 0.6800 f1_test: 0.6617 auc_test: 0.7055
### Seed: 710 Epoch: 697 acc_train: 0.7435 acc_test: 0.6820 f1_test: 0.6755 auc_test: 0.7047

In [29]:
args.act = nn.ReLU()
criterion = nn.BCELoss()
net = GCNNet(args)  
optimizer = optim.Adam(net.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)
net.cuda()


GCNNet(
  (blocks): ModuleList(
    (0): GCNBlock(
      (layers): ModuleList(
        (0): GCNLayer(
          (linear1): Linear(in_features=1248, out_features=400, bias=True)
          (linear2): Linear(in_features=1248, out_features=400, bias=True)
          (linear3): Linear(in_features=1248, out_features=400, bias=True)
          (linear4): Linear(in_features=1248, out_features=400, bias=True)
          (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
        )
      )
      (sc): GatedSkipConnection(
        (linear): Linear(in_features=1248, out_features=400, bias=False)
        (linear_coef_in): Linear(in_features=400, out_features=400, bias=True)
        (linear_coef_out): Linear(in_features=400, out_features=400, bias=True)
        (sigmoid): Sigmoid()
      )
    )
    (1): GCNBlock(
      (layers): ModuleList(
        (0): GCNLayer(
          (linear1): Linear(in_features=400, out_features=400, bias=True)


In [30]:
Adj = torch.from_numpy(Total_adj1).float().cuda()
train_y = torch.tensor(train_y, dtype=torch.long).cuda()
node_feature = torch.from_numpy(Node_feature1).float().cuda()
test_y = torch.tensor(test_y, dtype=torch.long).cuda()

In [ ]:
Adj = torch.from_numpy(Total_adj1).float().cuda()
node_feature = torch.from_numpy(Node_feature1).float().cuda()

In [ ]:
for epoch in range(args.epochs):
    train(epoch, args, criterion)

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:512: UserWarning: Using a target size (torch.Size([10849])) that is different to the input size (torch.Size([10849, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0001 acc_train: 0.5042 acc_test: 0.4900 f1_test: 0.4283 precision: 0.4872 recall: 0.3820
Epoch: 0002 acc_train: 0.5004 acc_test: 0.4980 f1_test: 0.4814 precision: 0.4979 recall: 0.4660
Epoch: 0003 acc_train: 0.4852 acc_test: 0.5310 f1_test: 0.5995 precision: 0.5231 recall: 0.7020
Epoch: 0004 acc_train: 0.4909 acc_test: 0.5330 f1_test: 0.5971 precision: 0.5250 recall: 0.6920
Epoch: 0005 acc_train: 0.5075 acc_test: 0.5490 f1_test: 0.5948 precision: 0.5400 recall: 0.6620
Epoch: 0006 acc_train: 0.5262 acc_test: 0.5520 f1_test: 0.5608 precision: 0.5500 recall: 0.5720
Epoch: 0007 acc_train: 0.5519 acc_test: 0.5530 f1_test: 0.5209 precision: 0.5612 recall: 0.4860
Epoch: 0008 acc_train: 0.5918 acc_test: 0.5510 f1_test: 0.4280 precision: 0.5895 recall: 0.3360
Epoch: 0009 acc_train: 0.6041 acc_test: 0.5680 f1_test: 0.3881 precision: 0.6650 recall: 0.2740
Epoch: 0010 acc_train: 0.6096 acc_test: 0.5590 f1_test: 0.3328 precision: 0.6832 recall: 0.2200
Epoch: 0011 acc_train: 0.6038 acc_test: 

Epoch: 0171 acc_train: 0.6611 acc_test: 0.6160 f1_test: 0.5212 precision: 0.6921 recall: 0.4180
Epoch: 0172 acc_train: 0.6633 acc_test: 0.6250 f1_test: 0.5455 precision: 0.6923 recall: 0.4500
Epoch: 0173 acc_train: 0.6636 acc_test: 0.6260 f1_test: 0.5483 precision: 0.6921 recall: 0.4540
Epoch: 0174 acc_train: 0.6618 acc_test: 0.6160 f1_test: 0.5188 precision: 0.6946 recall: 0.4140
Epoch: 0175 acc_train: 0.6611 acc_test: 0.6280 f1_test: 0.5592 precision: 0.6860 recall: 0.4720
Epoch: 0176 acc_train: 0.6590 acc_test: 0.6010 f1_test: 0.4838 precision: 0.6850 recall: 0.3740
Epoch: 0177 acc_train: 0.6622 acc_test: 0.6270 f1_test: 0.5658 precision: 0.6769 recall: 0.4860
Epoch: 0178 acc_train: 0.6628 acc_test: 0.6180 f1_test: 0.5213 precision: 0.6980 recall: 0.4160
Epoch: 0179 acc_train: 0.6651 acc_test: 0.6210 f1_test: 0.5406 precision: 0.6862 recall: 0.4460
Epoch: 0180 acc_train: 0.6637 acc_test: 0.6290 f1_test: 0.5630 precision: 0.6848 recall: 0.4780
Epoch: 0181 acc_train: 0.6630 acc_test: 

Epoch: 0341 acc_train: 0.6894 acc_test: 0.6380 f1_test: 0.5858 precision: 0.6845 recall: 0.5120
Epoch: 0342 acc_train: 0.6920 acc_test: 0.6370 f1_test: 0.5962 precision: 0.6717 recall: 0.5360
Epoch: 0343 acc_train: 0.6905 acc_test: 0.6380 f1_test: 0.5923 precision: 0.6778 recall: 0.5260
Epoch: 0344 acc_train: 0.6914 acc_test: 0.6390 f1_test: 0.5930 precision: 0.6796 recall: 0.5260
Epoch: 0345 acc_train: 0.6928 acc_test: 0.6390 f1_test: 0.5993 precision: 0.6733 recall: 0.5400
Epoch: 0346 acc_train: 0.6908 acc_test: 0.6400 f1_test: 0.5890 precision: 0.6862 recall: 0.5160
Epoch: 0347 acc_train: 0.6903 acc_test: 0.6430 f1_test: 0.6081 precision: 0.6740 recall: 0.5540
Epoch: 0348 acc_train: 0.6869 acc_test: 0.6320 f1_test: 0.5681 precision: 0.6875 recall: 0.4840
Epoch: 0349 acc_train: 0.6908 acc_test: 0.6400 f1_test: 0.6104 precision: 0.6651 recall: 0.5640
Epoch: 0350 acc_train: 0.6863 acc_test: 0.6330 f1_test: 0.5647 precision: 0.6939 recall: 0.4760
Epoch: 0351 acc_train: 0.6899 acc_test: 

Epoch: 0511 acc_train: 0.7130 acc_test: 0.6400 f1_test: 0.5955 precision: 0.6795 recall: 0.5300
Epoch: 0512 acc_train: 0.7153 acc_test: 0.6430 f1_test: 0.6064 precision: 0.6757 recall: 0.5500
Epoch: 0513 acc_train: 0.7162 acc_test: 0.6520 f1_test: 0.6250 precision: 0.6776 recall: 0.5800
Epoch: 0514 acc_train: 0.7132 acc_test: 0.6380 f1_test: 0.5849 precision: 0.6855 recall: 0.5100
Epoch: 0515 acc_train: 0.7150 acc_test: 0.6450 f1_test: 0.6243 precision: 0.6629 recall: 0.5900
Epoch: 0516 acc_train: 0.7125 acc_test: 0.6350 f1_test: 0.5691 precision: 0.6945 recall: 0.4820
Epoch: 0517 acc_train: 0.7144 acc_test: 0.6460 f1_test: 0.6250 precision: 0.6644 recall: 0.5900
Epoch: 0518 acc_train: 0.7129 acc_test: 0.6380 f1_test: 0.5810 precision: 0.6896 recall: 0.5020
Epoch: 0519 acc_train: 0.7158 acc_test: 0.6470 f1_test: 0.6176 precision: 0.6738 recall: 0.5700
Epoch: 0520 acc_train: 0.7167 acc_test: 0.6460 f1_test: 0.6110 precision: 0.6780 recall: 0.5560
Epoch: 0521 acc_train: 0.7166 acc_test: 

Epoch: 0681 acc_train: 0.7480 acc_test: 0.6310 f1_test: 0.5914 precision: 0.6625 recall: 0.5340
Epoch: 0682 acc_train: 0.7473 acc_test: 0.6390 f1_test: 0.5966 precision: 0.6759 recall: 0.5340
Epoch: 0683 acc_train: 0.7441 acc_test: 0.6420 f1_test: 0.6200 precision: 0.6606 recall: 0.5840
Epoch: 0684 acc_train: 0.7466 acc_test: 0.6370 f1_test: 0.5870 precision: 0.6807 recall: 0.5160
Epoch: 0685 acc_train: 0.7518 acc_test: 0.6410 f1_test: 0.6144 precision: 0.6636 recall: 0.5720
Epoch: 0686 acc_train: 0.7483 acc_test: 0.6390 f1_test: 0.6072 precision: 0.6659 recall: 0.5580
Epoch: 0687 acc_train: 0.7488 acc_test: 0.6400 f1_test: 0.5982 precision: 0.6768 recall: 0.5360
Epoch: 0688 acc_train: 0.7476 acc_test: 0.6440 f1_test: 0.6237 precision: 0.6614 recall: 0.5900
Epoch: 0689 acc_train: 0.7458 acc_test: 0.6340 f1_test: 0.5793 precision: 0.6811 recall: 0.5040
Epoch: 0690 acc_train: 0.7484 acc_test: 0.6400 f1_test: 0.6178 precision: 0.6584 recall: 0.5820
Epoch: 0691 acc_train: 0.7488 acc_test: 